# Лабораторная работа №5
## Выполнил студент группы БПИ2301

### Задание №1
Реализовать методы поиска подстроки в строке. Добавить возможность ввода строки и подстроки с клавиатуры. Предусмотреть возможность существования пробела. Реализовать возможность выбора опции чувствительности или нечувствительности к регистру. Оценить время работы каждого алгоритма поиска и сравнить его со временем работы стандартной функции поиска, используемой в выбранном языке программирования.

#### Алгоритм Кнута-Морриса-Пратта

In [96]:
def kmp_search(text, pattern):
    def build_lps(pattern):
        lps = [0] * len(pattern)
        length = 0
        i = 1
        while i < len(pattern):
            if pattern[i] == pattern[length]:
                length += 1
                lps[i] = length
                i += 1
            else:
                if length != 0:
                    length = lps[length - 1]
                else:
                    lps[i] = 0
                    i += 1
        return lps

    lps = build_lps(pattern)
    i = j = 0 # i - индекс в text, j - индекс в pattern
    positions = []
    while i < len(text):
        if pattern[j] == text[i]:
            i += 1
            j += 1
        if j == len(pattern):
            positions.append(i - j)
            j = lps[j - 1]
        elif i < len(text) and pattern[j] != text[i]:
            if j != 0:
                j = lps[j - 1]
            else:
                i += 1
    return positions

#### Упрощенный алгоритм Бойера-Мура

In [97]:
def simplified_boyer_moore(text, pattern):
    def bad_character(pattern):
        m = len(pattern)
        bad_char = {}
        for i in range(m - 1):
            bad_char[pattern[i]] = m - i - 1
        return bad_char

    m = len(pattern)
    n = len(text)
    bad_char = bad_character(pattern)
    positions = []

    s = 0
    while s <= n - m:
        j = m - 1
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            positions.append(s)
            s += 1
        else:
            shift = bad_char.get(text[s + j], m)
            s += shift

    return positions

Оценка времени работы алгоритмов

In [98]:
def default_search(text, pattern):
    positions = []
    index = text.find(pattern)
    while index != -1:
        positions.append(index)
        index = text.find(pattern, index + 1)
    return positions

In [99]:
import time

text = "abbcd asdbcf fbcde abbcdd bcgfdd acbde addbcde"
pattern = "cde"
case_sensitive = True

temp_text = text if case_sensitive else text.lower()
temp_pattern = pattern if case_sensitive else pattern.lower()

start = time.time()
kmp_result = kmp_search(temp_text, temp_pattern)
kmp_time = time.time() - start

start = time.time()
bm_result = simplified_boyer_moore(temp_text, temp_pattern)
bm_time = time.time() - start

start = time.time()
default_result = default_search(temp_text, temp_pattern)
default_time = time.time() - start

print(f"Результаты: kmp_result: {kmp_result}, bm_result: {bm_result}, default_result: {default_result}\n")
print(f"Алгоритм Кнута-Морриса-Пратта: {kmp_time:.7f}")
print(f"Упрощенный алгоритм Бойера-Мура: {bm_time:.7f}")
print(f"Стандартный поиск: {default_time:.7f}")

Результаты: kmp_result: [15, 43], bm_result: [15, 43], default_result: [15, 43]

Алгоритм Кнута-Морриса-Пратта: 0.0000780
Упрощенный алгоритм Бойера-Мура: 0.0000384
Стандартный поиск: 0.0000386


### Задание №2
Написать программу, определяющую, является ли данное расположение «решаемым», то есть можно ли из него за конечное число шагов перейти к правильному. Если это возможно, то необходимо найти хотя бы одно решение - последовательность движений, после которой числа будут расположены в правильном порядке.

Входные данные: массив чисел, представляющий собой расстановку в порядке «слева направо, сверху вниз». Число 0 обозначает пустое поле.
Например, массив [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0] представляет собой «решенную» позицию элементов.

Выходные данные: если решения нет, то функция должна вернуть пустой массив []. Если решение есть, то необходимо представить решение — для каждого шага записывается номер передвигаемого на данном шаге элемента.

In [ ]:
from collections import deque

def is_solvable(state):
    inv_count = 0
    size = int(len(state) ** 0.5)
    for i in range(len(state)):
        for j in range(i + 1, len(state)):
            if state[i] != 0 and state[j] != 0 and state[i] > state[j]:
                inv_count += 1

    if size % 2 == 1:
        return inv_count % 2 == 0
    else:
        zero_row = state.index(0) // size
        row_from_bottom = size - zero_row
        return (inv_count + row_from_bottom) % 2 == 1


def get_neighbors(state):
    neighbors = []
    n = int(len(state) ** 0.5)
    zero_pos = state.index(0)
    row, col = divmod(zero_pos, n)

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

    for dr, dc in directions:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < n and 0 <= new_col < n:
            new_pos = new_row * n + new_col
            new_state = list(state)
            new_state[zero_pos], new_state[new_pos] = new_state[new_pos], new_state[zero_pos]
            neighbors.append((new_state, new_pos))

    return neighbors


def solve_puzzle(start_state, max_depth=20):
    n = int(len(start_state) ** 0.5)
    goal_state = list(range(1, n*n)) + [0]

    if start_state == goal_state:
        return []
    
    if not is_solvable(start_state):
        return []

    queue = deque([(start_state, start_state.index(0), [])])
    visited = set()
    visited.add(tuple(start_state))

    while queue:
        current_state, zero_pos, path = queue.popleft()

        if len(path) > max_depth:
            continue

        for neighbor, new_pos in get_neighbors(current_state):
            if tuple(neighbor) not in visited:
                visited.add(tuple(neighbor))
                new_path = path + [current_state[new_pos]]
                if neighbor == goal_state:
                    return new_path
                queue.append((neighbor, new_pos, new_path))

    return []

In [116]:
start_state = [1, 2, 3, 4,
               5, 6, 7, 8,
               9, 10, 11, 12,
               13, 14, 15, 0]

start_time = time.time()
solution = solve_puzzle(start_state)
end_time = time.time()

print("#1 Solution:", solution)
print("   Time:", end_time - start_time)

start_state = [1, 2, 3, 4,
               5, 6, 7, 8,
               9, 10, 11, 12,
               13, 14, 0, 15]

start_time = time.time()
solution = solve_puzzle(start_state)
end_time = time.time()

print("#2 Solution:", solution)
print("   Time:", end_time - start_time)

start_state = [1, 2, 3, 4,
               5, 6, 7, 8,
               9, 10, 12, 11,
               13, 15, 14, 0]
start_time = time.time()
solution = solve_puzzle(start_state)
end_time = time.time()

print("#3 Solution:", solution)
print("   Time:", end_time - start_time)

#1 Solution: []
   Time: 4.4345855712890625e-05
#2 Solution: [15]
   Time: 7.462501525878906e-05
#3 Solution: [14, 12, 11, 14, 12, 15, 10, 11, 14, 12, 15, 14, 11, 10, 14, 15]
   Time: 0.3974592685699463


### Вывод

В лабораторной работе реализованы алгоритмы поиска подстроки (Кнута-Морриса-Пратта и упрощённый Бойер-Мур) с учётом регистра и пробелов, а также решена задача «Пятнашек» с проверкой решаемости и поиском решения через поиск в ширину. Работа помогла лучше понять алгоритмы поиска и моделирование состояний для решения задач с оптимальным путём.